In [42]:
import sys
!{sys.executable} -m pip install lxml

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [43]:
import requests
import lxml.html as lh
import pandas as pd
import plotly.graph_objs as plotly
import numpy as np
import datetime

In [44]:
#Let's see what the output for one month looks like:
year = '2020'
month = '01' #adapt for for loop
state = 'massachusetts'
location = 'great-hill-ma'

#This code (next 3 lines) is adapted from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
url='https://www.usharbors.com/harbor/'+state+'/'+location+'/tides/?tide='+year+'-'+month+'#monthly-tide-chart'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#save the raw html rows as "table"
table = tr_elements

In [45]:
#print the lengths of the first few rows to see where the actual tide chart starts
print("\nThe number of columns in the first twelve rows are:")
print([len(T) for T in table[:12]])

#check the output of one row
print("\nSample row output:")
row = 10
for col in range(len(table[row])):
    print(table[row][col].text_content())
print("Doesn't look like we need the last 3 columns. So we'll just grab the first 12.")
number_of_columns = 12
    
#Check how many rows we have
print("\nThe number of rows is: " + str(len(table)))


The number of columns in the first twelve rows are:
[6, 10, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

Sample row output:
9
Thu
6:40
4.7
7:04
3.7


12:17
0.1
7:07
4:31

9
Thu
Doesn't look like we need the last 3 columns. So we'll just grab the first 12.

The number of rows is: 35


In [46]:
#Function which scrapes the URL for the current month
def get_month_table(month):
    if month > 9: #month as a string to use for the URL def
        month_str = str(month)
    else: 
        month_str = '0'+str(month)
    #This code (next 3 lines) is adapted from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    url='https://www.usharbors.com/harbor/'+state+'/'+location+'/tides/?tide='+year+'-'+month_str+'#monthly-tide-chart'
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    #save the raw html rows as "table"
    table = tr_elements
    return(table)

In [47]:
def find_first_row(table):
    for row in range(len(table)):
        if table[row][0].text_content() == '1':
            return(row)

In [48]:
def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [49]:
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

In [50]:
year_str = str(year)

#Empty df to store the data:
df = []

for month in range(1,13):
    month_str_display = months[month-1]
    table = get_month_table(month) #get the data for this month

    #find the first row
    row = find_first_row(table)

    #while the first entry of the row is less than the number of days in the month
    while isInt(table[row][0].text_content()):
        row_data = ['']*number_of_columns
        for col in range(number_of_columns):
            row_data[col] = table[row][col].text_content()
        row_data.insert(0,month_str_display)
        df.append(row_data)
        row+=1

In [51]:
print("The number of rows is "+str(len(df))+", this number should match the number of days in the given year, so 365 or 366")


The number of rows is 366, this number should match the number of days in the given year, so 365 or 366


In [52]:
column_names = ['Month','Date','Day','high_1','height_hi_1','high_2','height_hi_2','low_1','height_low_1','low_2','height_low_2','sunrise','sunset']
data = pd.DataFrame(df,columns=column_names)

In [53]:
data

,Month,Date,Day,high_1,height_hi_1,high_2,height_hi_2,low_1,height_low_1,low_2,height_low_2,sunrise,sunset
0,January,1,Wed,12:00,3.6,12:20,3.6,4:57,0.7,5:21,0.6,7:08,4:24
1,January,2,Thu,12:53,3.6,1:14,3.4,5:45,0.8,6:05,0.7,7:08,4:25
2,January,3,Fri,1:44,3.7,2:04,3.3,6:43,1.0,6:57,0.7,7:08,4:25
3,January,4,Sat,2:32,3.8,2:55,3.2,7:51,1.0,7:53,0.8,7:08,4:26
4,January,5,Sun,3:23,3.9,3:48,3.2,9:04,0.9,8:50,0.7,7:08,4:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,December,27,Sun,5:56,4.2,6:20,3.4,12:18 PM,0.6,11:26,0.6,7:07,4:20
362,December,28,Mon,6:37,4.3,7:01,3.5,12:23 PM,0.5,11:59,0.4,7:07,4:21
363,December,29,Tue,7:17,4.4,7:40,3.6,,,12:50,0.4,7:07,4:22
364,December,30,Wed,7:56,4.5,8:20,3.6,12:35,0.3,1:26,0.3,7:08,4:23


In [54]:
data.to_csv('tide-chart-'+location+'-'+year_str+'.csv',index=False)

In [55]:
#one time imports for plotting
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

#test the plotting 
x = np.array([1, 2, 3, 4, 5])
y = np.array([1, 3, 2, 3, 1])
fig=go.Scatter(x=x, y=y, name="spline",line_shape='spline')
py.iplot([fig],filename='test')

In [56]:
high1 = pd.melt(data,id_vars=['Month','Date','Day','height_hi_1'],value_vars=['high_1'],var_name='event', value_name='time')
high2 = pd.melt(data,id_vars=['Month','Date','Day','height_hi_2'],value_vars=['high_2'],var_name='event', value_name='time')
low1 = pd.melt(data,id_vars=['Month','Date','Day','height_low_1'],value_vars=['low_1'],var_name='event', value_name='time')
low2 = pd.melt(data,id_vars=['Month','Date','Day','height_low_2'],value_vars=['low_2'],var_name='event', value_name='time')
sunrise = pd.melt(data,id_vars=['Month','Date','Day'],value_vars=['sunrise'],var_name='event', value_name='time')
sunset = pd.melt(data,id_vars=['Month','Date','Day'],value_vars=['sunset'],var_name='event', value_name='time')

In [57]:
high1 = high1.rename(columns={"height_hi_1": "height"})
high2 = high2.rename(columns={"height_hi_2": "height"})
low1 = low1.rename(columns={"height_low_1": "height"})
low2 = low2.rename(columns={"height_low_2": "height"})
sunrise['height'] = np.nan
sunset['height'] = np.nan

In [58]:
data = (pd.concat([high1, high2, low1, low2, sunrise, sunset])
         .sort_index()
         .reset_index())

In [59]:
data = data[data['time']!='']
data

,index,Month,Date,Day,height,event,time
0,0,January,1,Wed,3.6,high_1,12:00
1,0,January,1,Wed,0.6,low_2,5:21
2,0,January,1,Wed,3.6,high_2,12:20
3,0,January,1,Wed,NaN,sunrise,7:08
4,0,January,1,Wed,NaN,sunset,4:24
...,...,...,...,...,...,...,...
2191,365,December,31,Thu,0.1,low_2,2:07
2192,365,December,31,Thu,3.7,high_2,9:01
2193,365,December,31,Thu,4.5,high_1,8:36
2194,365,December,31,Thu,NaN,sunrise,7:08


In [60]:
import re
for row in range(len(data)):
    event = data['event'].iloc[row]
    time = data['time'].iloc[row]
    if (event == 'high_1' or event == 'low_1' or event == 'sunrise'): #morning events
        if not re.search('PM',str(time)):
            time = time + ' AM'
        time = pd.to_datetime(time).strftime('%H:%M %p')
    if (event == 'high_2' or event == 'low_2' or event == 'sunset'): #evening events
        if not re.search('AM',str(time)):
            time = time + ' PM'
        time = pd.to_datetime(time).strftime('%H:%M %p')
    data['time'].iloc[row] = time

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
data

,index,Month,Date,Day,height,event,time
0,0,January,1,Wed,3.6,high_1,00:00 AM
1,0,January,1,Wed,0.6,low_2,17:21 PM
2,0,January,1,Wed,3.6,high_2,12:20 PM
3,0,January,1,Wed,NaN,sunrise,07:08 AM
4,0,January,1,Wed,NaN,sunset,16:24 PM
...,...,...,...,...,...,...,...
2191,365,December,31,Thu,0.1,low_2,14:07 PM
2192,365,December,31,Thu,3.7,high_2,21:01 PM
2193,365,December,31,Thu,4.5,high_1,08:36 AM
2194,365,December,31,Thu,NaN,sunrise,07:08 AM


In [62]:
data['full_date'] = data['Month']+'-'+data['Date']+'-'+year+' '+data['time']
data['full_date'] = pd.to_datetime(data['full_date'], format='%B-%d-%Y %H:%M %p')

In [63]:
data['week_num'] = np.nan
for row in range(len(data)):
    data['week_num'].iloc[row] = data['full_date'].iloc[row].isocalendar()[1]
    if data['Day'].iloc[row] == 'Sun':
        data['week_num'].iloc[row] += 1

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [64]:
data

,index,Month,Date,Day,height,event,time,full_date,week_num
0,0,January,1,Wed,3.6,high_1,00:00 AM,2020-01-01 00:00:00,1.0
1,0,January,1,Wed,0.6,low_2,17:21 PM,2020-01-01 17:21:00,1.0
2,0,January,1,Wed,3.6,high_2,12:20 PM,2020-01-01 12:20:00,1.0
3,0,January,1,Wed,NaN,sunrise,07:08 AM,2020-01-01 07:08:00,1.0
4,0,January,1,Wed,NaN,sunset,16:24 PM,2020-01-01 16:24:00,1.0
...,...,...,...,...,...,...,...,...,...
2191,365,December,31,Thu,0.1,low_2,14:07 PM,2020-12-31 14:07:00,53.0
2192,365,December,31,Thu,3.7,high_2,21:01 PM,2020-12-31 21:01:00,53.0
2193,365,December,31,Thu,4.5,high_1,08:36 AM,2020-12-31 08:36:00,53.0
2194,365,December,31,Thu,NaN,sunrise,07:08 AM,2020-12-31 07:08:00,53.0


In [65]:
# minutes from the previous sunday at midnight

In [66]:
now = datetime.datetime.now() # current time
data['minutes_until'] = np.nan
for row in range(len(data)): # calculate the number of minutes until each time
    delta = data['full_date'].iloc[row] - now
    data['minutes_until'].iloc[row] = pd.Timedelta(delta).total_seconds()/60

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [67]:
data.loc[1,:]

index                              0
Month                        January
Date                               1
Day                              Wed
height                           0.6
event                          low_2
time                        17:21 PM
full_date        2020-01-01 17:21:00
week_num                           1
minutes_until                -502668
Name: 1, dtype: object